In [1]:
import numpy as np


In [14]:
class NeuralNetwork():
    def __init__(self,inputs,outputs,outputActivation="softmax",epochs=100,learning_rate=0.001,validation=[]):
        self.inputs = inputs
        self.outputs = outputs
        self.layers = []
        self.weights = []
        self.biases = []
        self.actual_outputs= []
        self.activations = []
        self.deltas = []
        self.delta_weights = []
        self.delta_biases = []
        self.outputActivation=outputActivation, 
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.validation=validation
    def add_layer(self,layer_num,activation='sigmoid'):
        self.layers.append({
            "index" : len(self.layers),
            "layer_num":layer_num,
            "activation": activation
        })
    
    def initialize_weights(self):
        if hasattr(self.inputs[0], "__len__"):
            inputWeight = len(self.inputs[0])
        else:
            inputWeight = 1
            
        if hasattr(self.outputs[0], "__len__"):
            outputWeight = len(self.outputs[0])
        else:
            outputWeight = 1
        for index,eachLayer in enumerate(self.layers):
            if index==0:
                self.weights.append(np.random.rand(eachLayer["layer_num"],inputWeight))
                self.biases.append(np.random.rand(eachLayer["layer_num"],1))                
            else:
                self.weights.append(np.random.rand(self.layers[index]["layer_num"],self.layers[index-1]["layer_num"]))
                self.biases.append(np.random.rand(eachLayer["layer_num"],1))
                
                if index==len(self.layers)-1:
                    self.weights.append(np.random.rand(outputWeight,self.layers[index]["layer_num"]))
                    self.biases.append(np.random.rand(outputWeight,1))
                    
    def sigmoid(self,x,derivative=False):
        if derivative:
            return self.sigmoid(x)*(1-self.sigmoid(x))
        else:
            return 1/(1+np.exp(-x))
                
    def softmax(self,X):
        eachV = []
        for eachX in X:
            eachV.append(np.exp(eachX)/np.sum(np.exp(X)))
        return np.array(eachV)
                                    
    def feed_forward(self):
        
        for eachIndex in range(len(self.inputs)):
            activations = []
            actual_outputs = []
            for index,eachLayer in enumerate(self.layers):
                if index==0:
                    if eachLayer["activation"]=="sigmoid":
                        actual_outputs.append(np.dot(self.weights[index],self.inputs[eachIndex]) + self.biases[index])
                        activations.append(self.sigmoid(actual_outputs[-1]))
                else:
                    if eachLayer["activation"]=="sigmoid":
                        actual_outputs.append(np.dot(self.weights[index],activations[-1]) + self.biases[index])
                        activations.append(self.sigmoid(np.dot(self.weights[index],activations[-1]) + self.biases[index]))
                    if index==len(self.layers)-1:
                        if self.outputActivation[0]=="softmax":
                            actual_outputs.append(np.dot(self.weights[-1],activations[-1])+self.biases[-1])
                            activations.append(self.softmax(np.dot(self.weights[-1],activations[-1])+self.biases[-1]))
            self.activations.append(activations[-1])
            self.actual_outputs.append(actual_outputs[-1])
            activations= []
            actual_outputs=[]

                    
    def loss(self,loss_type="cross_entropy"):
        if loss_type=="mse":
            return 1/2*(np.square(self.outputs-np.array(self.activations[-1])))
        elif loss_type== "cross_entropy":
            return np.sum(self.outputs*np.log(np.array(self.activations[-1])) + (1-self.outputs)*np.log(1-self.activations[-1]))
        else:
            raise Exception('UNKNOWN ERROR')
            
    def backpropagate(self):
        for eachIndex in range(len(self.inputs)):
            num = np.array(self.activations[eachIndex]-self.outputs[eachIndex]) 
            deltas = []
            for index in range(len(self.weights))[::-1]:
                if index==len(self.weights)-1:
                    delta = num*self.sigmoid(self.actual_outputs[eachIndex],derivative=True)
                    deltas.append(delta) 
                else:
                    delta = np.dot(self.weights[index+1].transpose(),deltas[-1])*self.sigmoid(self.actual_outputs[eachIndex],derivative=True)
                    deltas.append(delta)
                    if index==0:
                        delta = np.dot(self.weights[index].transpose(),deltas[-1])*self.sigmoid(self.inputs[eachIndex],derivative=True)
                        deltas.append(delta)
                        print(np.array(delta).shape)
            
            if(eachIndex==0):
                break           
            self.delta_weights.append(delta*self.inputs[eachIndex])
            self.delta_biases.append(delta)
                    
        return np.array(self.delta_weights),np.array(self.delta_biases)
    
    def GradientDescent(self):
        self.initialize_weights()
        for i in range(self.epochs):
            self.feed_forward()
            d_w,d_b = self.backpropagate()
            self.weights = self.weights - self.learning_rate*d_w*self.weights
            self.biases = self.biases-self.learning_rate*d_b*self.biases
            prediction = np.dot(self.weights,self.validation)+self.biases
            print("Epoch Number {} Loss {} prediction ={} ".format(i+1,self.loss(loss_type='mse'),prediction))
        
        
    
    

In [15]:
X =[]
Y= []
for i in range(10000):
    choiceXFirst = np.random.choice([0,1])
    choiceXSecond = np.random.choice([0,1])
    X.append([choiceXFirst,choiceXSecond])
    if (choiceXFirst==1 and choiceXSecond==1):
        Y.append(1)
    else:
        Y.append(0)
X =np.array(X)
Y = np.array(Y)
X = np.reshape(X,(X.shape[0],X.shape[1],1))
Y = np.reshape(Y,(Y.shape[0],1))

In [16]:
nn = NeuralNetwork(X,Y)
nn.add_layer(10)
nn.add_layer(20)
nn.initialize_weights()
nn.feed_forward()
nn.backpropagate()
nn.GradientDescent()

(2, 1)


ValueError: shapes (20,1) and (2,1) not aligned: 1 (dim 1) != 2 (dim 0)